In [1]:
import pandas as pd
import numpy as np
import os
import acquire_nick as get
import prepare_nick as prep
import re
import wrangle as w

In [5]:
df = w.wrangle_df()

Using cached CSV


In [7]:
df.columns

Index(['id', 'title', 'success', 'success_rating', 'genres', 'cast_actor_1',
       'cast_actor_2', 'cast_actor_3', 'total_n_cast', 'budget', 'revenue',
       'profit_amount', 'vote_average', 'vote_count', 'production_companies',
       'production_countries', 'overview', 'popularity', 'runtime',
       'profitable', 'release_date', 'release_year', 'runtime.1', 'imdb_id'],
      dtype='object')